In [48]:
import pandas as pd
import pickle

In [49]:
data = pd.read_pickle(r'C:\Users\PIYUSH\Downloads\merged_training.pkl')

In [50]:
data.head()

,text,emotions
27383,i feel awful about it too because it s my job ...,sadness
110083,im alone i feel awful,sadness
140764,ive probably mentioned this before but i reall...,joy
100071,i was feeling a little low few days back,sadness
2837,i beleive that i am much more sensitive to oth...,love


In [51]:
data=data.iloc[:80000]

In [52]:
data.shape

(80000, 2)

In [53]:
data.columns = ['content', 'sentiments']

In [54]:
data['sentiments'].unique()

array(['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
      dtype=object)

In [55]:
import re
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [56]:
#Making all letters lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation, Symbols
data['content'] = data['content'].str.replace('[^\w\s]',' ')
#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Lemmatisation
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'] = data['content'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))


In [57]:
# # Code to find the top 10,000 rarest words appearing in the data
# freq = pd.Series(' '.join(data['content']).split()).value_counts()[-10000:]
# # Removing all those rarely appearing words from the data
# freq = list(freq.index)
# data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [58]:
data.head()

,content,sentiments
27383,feel awful job get position succeed happen,sadness
110083,im alone feel awful,sadness
140764,ive probably mentioned really feel proud actua...,joy
100071,feeling little low day back,sadness
2837,beleive much sensitive people feeling tend com...,love


enthusiasm=happy=fun
empty=neutral
boredom=sadness

In [14]:
data.head()

,content,sentiments
27383,feel awful job get position succeed happen,sadness
110083,im alone feel awful,sadness
140764,ive probably mentioned really feel proud actua...,joy
100071,feeling little low day back,sadness
2837,beleive much sensitive people feeling tend com...,love


In [59]:
#Encoding output labels 'sadness' as '1' & 'happiness' as '0'
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiments.values)
# Splitting into training and testing data in 90:10 ratio

In [60]:
y

array([4, 4, 2, ..., 2, 4, 4], dtype=int64)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data.content.values, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)


In [62]:
# Extracting TF-IDF parameters
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, analyzer='word',ngram_range=(1,3))
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.fit_transform(X_val)

In [63]:
X_train_tfidf=X_train_tfidf.toarray()

In [64]:
X_val_tfidf=X_val_tfidf.toarray()

In [65]:
X_val_tfidf=X_val_tfidf.reshape(X_val_tfidf.shape[0],1,X_val_tfidf.shape[1])

In [66]:
X_train_tfidf=X_train_tfidf.reshape(X_train_tfidf.shape[0],1,X_train_tfidf.shape[1])

In [67]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_train.shape

(72000, 6)

In [68]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [74]:
classifier = Sequential()
classifier.add(LSTM(64,input_shape=(1,1000),return_sequences=True))
classifier.add(Dropout(0.4))
classifier.add(LSTM(64))
classifier.add(Dropout(0.3))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 1, 64)             272640    
_________________________________________________________________
dropout_12 (Dropout)         (None, 1, 64)             0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
_________________________________________________________________
activation_4 (Activation)    (None, 6)                 0         
Total params: 306,054
Trainable params: 306,054
Non-trainable params: 0
________________________________________________

In [75]:

classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
classifier.fit(X_train_tfidf,y_train,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Train on 64800 samples, validate on 7200 samples
Epoch 1/40
64800/64800 [==============================] - 16s 247us/sample - loss: 0.7461 - acc: 0.7271 - val_loss: 0.3905 - val_acc: 0.8546
Epoch 2/40
64800/64800 [==============================] - 13s 193us/sample - loss: 0.4087 - acc: 0.8524 - val_loss: 0.3610 - val_acc: 0.8576
Epoch 3/40
64800/64800 [==============================] - 13s 193us/sample - loss: 0.3793 - acc: 0.8585 - val_loss: 0.3540 - val_acc: 0.8597
Epoch 4/40
64800/64800 [==============================] - 13s 194us/sample - loss: 0.3652 - acc: 0.8609 - val_loss: 0.3544 - val_acc: 0.8607
Epoch 5/40
64800/64800 [==============================] - 13s 196us/sample - loss: 0.3507 - acc: 0.8645 - val_loss: 0.3486 - val_acc: 0.8574
Epoch 6/40
64800/64800 [==============================] - 12s 193us/sample - loss: 0.3464 - acc: 0.8641 - val_loss: 0.3479 - val_acc: 0.8576
Epoch 7/40
64800/64800 [==============================] - 13s 195us/sample - loss: 0.3381 - acc: 0.8673 -

In [82]:
y_prob = classifier.predict(X_val_tfidf) 
y_classes = y_prob.argmax(axis=-1)

In [83]:
y_val = y_val.argmax(axis=-1)
y_val

30